# 🔧 01 - Configuração e Setup do Sistema

## 📖 Visão Geral

Este notebook contém as configurações essenciais para o sistema de análise de nanorevestimentos e tintas. É o primeiro notebook a ser executado no pipeline.

### 🎯 Responsabilidades

- ✅ Configuração da API do Google Gemini
- ✅ Definição de padrões e bases de conhecimento
- ✅ Configuração de rate limiting e checkpoints
- ✅ Validação do ambiente e dependências
- ✅ Inicialização de variáveis globais

### 📦 Dependências

- google-generativeai
- pandas
- pathlib
- os

### 🔗 Próximo Notebook

Após configurar este notebook, execute: `02_carregamento_dados.ipynb`

In [1]:
# 📦 Instalação e imports das dependências
!pip install google-generativeai pandas numpy matplotlib seaborn -q

import pandas as pd
import google.generativeai as genai
import time
import random
import os
import json
from pathlib import Path
from typing import List, Optional, Dict, Any
from datetime import datetime

print("✅ Dependências carregadas com sucesso!")

✅ Dependências carregadas com sucesso!


In [2]:
# Helper functions para carregar e salvar config_sistema.json

def load_config(file_path: str) -> Dict[str, Any]:
    """Carrega o arquivo de configuração JSON."""
    if Path(file_path).exists():
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                config_data = json.load(f)
            print(f"✅ Configuração carregada de: {file_path}")
            return config_data
        except Exception as e:
            print(f"⚠️ Erro ao carregar {file_path}: {e}. Usando configuração padrão.")
            return {}
    else:
        print(f"ℹ️ Arquivo de configuração não encontrado em {file_path}. Uma nova configuração será criada.")
        return {}

def save_config(config_data: Dict[str, Any], file_path: str):
    """Salva o dicionário de configuração em um arquivo JSON."""
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(config_data, f, indent=2, ensure_ascii=False)
        print(f"💾 Configuração salva com sucesso em: {file_path}")
    except Exception as e:
        print(f"❌ Erro ao salvar configuração em {file_path}: {e}")

In [3]:
# 🔑 Configuração da API do Google Gemini

def validar_api_key(api_key: str) -> bool:
    """Valida formato básico da API key do Gemini"""
    if not api_key or len(api_key.strip()) < 20:
        return False
    if not api_key.startswith('AIza'):
        return False
    return True

def carregar_api_key() -> Optional[str]:
    """Carrega API key de diferentes fontes com validação robusta"""
    
    print("🔑 Carregando API key...")
    
    # Tentar carregar de diferentes arquivos
    caminhos_possiveis = ['api_key', 'api_key.txt', '.api_key', 'gemini_api_key']
    
    for caminho in caminhos_possiveis:
        if os.path.exists(caminho):
            try:
                with open(caminho, 'r', encoding='utf-8') as f:
                    api_key = f.read().strip()
                if validar_api_key(api_key):
                    print(f"✅ API key carregada de: {caminho}")
                    return api_key
                else:
                    print(f"❌ API key inválida em: {caminho}")
            except Exception as e:
                print(f"⚠️ Erro ao ler {caminho}: {e}")
    
    # Se não encontrou, orientar o usuário
    print("\n🔑 API key não encontrada. Vamos configurá-la agora!")
    print("\n📋 Para obter sua API key GRATUITA:")
    print("   1. Acesse: https://makersuite.google.com/app/apikey")
    print("   2. Faça login com sua conta Google")
    print("   3. Clique em 'Create API Key'")
    print("   4. Copie a chave gerada (começa com AIza...)")
    
    api_key = input("\n🔑 Cole sua API key aqui: ").strip()
    
    if validar_api_key(api_key):
        try:
            with open('api_key', 'w', encoding='utf-8') as f:
                f.write(api_key)
            print("✅ API key salva em 'api_key'")
            return api_key
        except Exception as e:
            print(f"⚠️ Erro ao salvar: {e}")
            return api_key
    else:
        print("❌ API key inválida")
        return None

def testar_conexao_gemini(api_key: str) -> bool:
    """Testa conexão com API do Gemini"""
    print("🧪 Testando conexão com Gemini AI...")
    
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-2.0-flash-exp')
        
        # Teste simples
        response = model.generate_content("Teste de conexão")
        
        if response and response.text:
            print("✅ Conexão com Gemini AI estabelecida com sucesso!")
            return True
        else:
            print("❌ Resposta vazia da API")
            return False
            
    except Exception as e:
        print(f"❌ Erro na conexão: {str(e)}")
        return False

# Executar configuração da API
api_key = carregar_api_key()

if api_key:
    conexao_ok = testar_conexao_gemini(api_key)
    if conexao_ok:
        print("🎉 API configurada e funcional!")
        API_CONFIGURADA = True
    else:
        print("⚠️ API configurada mas com problemas de conexão")
        API_CONFIGURADA = False
else:
    print("❌ Não foi possível configurar a API")
    API_CONFIGURADA = False

🔑 Carregando API key...
✅ API key carregada de: api_key.txt
🧪 Testando conexão com Gemini AI...
✅ Conexão com Gemini AI estabelecida com sucesso!
🎉 API configurada e funcional!
✅ Conexão com Gemini AI estabelecida com sucesso!
🎉 API configurada e funcional!


In [4]:
# 🧬 Base de Conhecimento - Nanomateriais Especializados

# Padrões regex para identificação de nanomateriais
PATTERNS_NANOMATERIAIS = {
    'oxidos_metalicos': [
        'TiO2', 'ZnO', 'Al2O3', 'SiO2', 'Fe2O3', 'CeO2', 'Cr2O3',
        'titanium dioxide', 'zinc oxide', 'aluminum oxide', 'silica',
        'iron oxide', 'cerium oxide', 'chromium oxide'
    ],
    'carbono': [
        'graphene', 'grafeno', 'CNT', 'carbon nanotube', 'carbon fiber',
        'fullerene', 'carbon black', 'carbon dot', 'graphene oxide',
        'reduced graphene oxide', 'rGO', 'MWCNT', 'SWCNT'
    ],
    'metais_nobres': [
        'Ag', 'Au', 'Pt', 'Pd', 'silver', 'gold', 'platinum', 'palladium',
        'silver nanoparticle', 'gold nanoparticle', 'AgNP', 'AuNP'
    ],
    'polimeros': [
        'PMMA', 'PVA', 'PVP', 'PEG', 'chitosan', 'alginate',
        'polyurethane', 'epoxy', 'acrylic', 'polyethylene',
        'polypropylene', 'nylon'
    ],
    'ceramicos': [
        'hydroxyapatite', 'zirconia', 'alumina', 'silicon carbide',
        'silicon nitride', 'boron nitride', 'titanium carbide'
    ],
    'compositos': [
        'nanocomposite', 'hybrid material', 'core-shell',
        'layered material', 'intercalated', 'exfoliated'
    ]
}

# Padrões para palavras-chave funcionais
PATTERNS_PALAVRAS_CHAVE = {
    'coating_terms': [
        'coating', 'paint', 'revestimento', 'tinta', 'surface treatment',
        'protective coating', 'functional coating', 'thin film'
    ],
    'propriedades_funcionais': [
        'anticorrosive', 'antimicrobial', 'self-cleaning', 'superhydrophobic',
        'UV protection', 'thermal barrier', 'electrical conductivity',
        'magnetic', 'photocatalytic', 'antifouling'
    ],
    'aplicacoes': [
        'automotive', 'marine', 'aerospace', 'biomedical', 'electronics',
        'construction', 'textile', 'packaging', 'energy storage'
    ],
    'tecnicas_preparacao': [
        'sol-gel', 'spray coating', 'dip coating', 'spin coating',
        'electrodeposition', 'plasma spraying', 'CVD', 'PVD',
        'layer-by-layer', 'self-assembly'
    ]
}

print("🧬 Base de conhecimento carregada:")
print(f"  📊 {len(PATTERNS_NANOMATERIAIS)} categorias de nanomateriais")
print(f"  🔑 {len(PATTERNS_PALAVRAS_CHAVE)} categorias de palavras-chave")

# Contar total de termos
total_nano_terms = sum(len(terms) for terms in PATTERNS_NANOMATERIAIS.values())
total_keyword_terms = sum(len(terms) for terms in PATTERNS_PALAVRAS_CHAVE.values())

print(f"  🎯 {total_nano_terms} termos de nanomateriais específicos")
print(f"  🎯 {total_keyword_terms} palavras-chave funcionais")

🧬 Base de conhecimento carregada:
  📊 6 categorias de nanomateriais
  🔑 4 categorias de palavras-chave
  🎯 64 termos de nanomateriais específicos
  🎯 37 palavras-chave funcionais


In [5]:
# ⚙️ Configurações do Sistema

# Configurações de rate limiting para API
CONFIG = {
    'delay_between_requests': 2.5,  # segundos entre requisições
    'max_requests_per_minute': 25,  # limite seguro
    'max_requests_per_day': 1400,   # limite diário
    'timeout_request': 30,          # timeout por requisição
    'max_retries': 3,              # tentativas em caso de erro
    'backoff_factor': 2,           # fator de backoff exponencial
}

# Configurações de checkpoint
CHECKPOINT_CONFIG = {
    'interval': 25,                # salvar checkpoint a cada N artigos
    'directory': 'checkpoints/',   # diretório para checkpoints
    'prefix': 'checkpoint_',       # prefixo dos arquivos
    'max_files': 10,              # máximo de arquivos de checkpoint
}

# Configurações de caminhos
PATHS = {
    'data_raw': '/home/delon/Modelos/modeloCenanoInk/data/raw/',
    'excel_files': '/home/delon/Modelos/modeloCenanoInk/data/raw/arquivos_excel_artigo_cienciometrico/',
    'processed': '/home/delon/Modelos/modeloCenanoInk/data/processed/',
    'results': '/home/delon/Modelos/modeloCenanoInk/results/',
    'notebooks': '/home/delon/Modelos/modeloCenanoInk/notebooks/'
}

# Criar diretórios se não existirem
for path_name, path_value in PATHS.items():
    Path(path_value).mkdir(parents=True, exist_ok=True)

# Configurações de análise
ANALYSIS_CONFIG = {
    'min_abstract_length': 50,     # tamanho mínimo do abstract
    'score_threshold_high': 5,     # threshold para relevância alta
    'score_threshold_medium': 2,   # threshold para relevância média
    'use_title_boost': True,       # dar peso extra para palavras no título
    'title_boost_factor': 2,       # fator de multiplicação para título
}

print("⚙️ Configurações do sistema carregadas:")
print(f"  🔄 Rate limiting: {CONFIG['delay_between_requests']}s entre requisições")
print(f"  💾 Checkpoints: a cada {CHECKPOINT_CONFIG['interval']} artigos")
print(f"  📁 Diretórios: {len(PATHS)} caminhos configurados")
print(f"  🎯 Análise: thresholds {ANALYSIS_CONFIG['score_threshold_medium']}/{ANALYSIS_CONFIG['score_threshold_high']}")

⚙️ Configurações do sistema carregadas:
  🔄 Rate limiting: 2.5s entre requisições
  💾 Checkpoints: a cada 25 artigos
  📁 Diretórios: 5 caminhos configurados
  🎯 Análise: thresholds 2/5


In [6]:
# ⚙️ Configurações do Sistema

config_file_path = PATHS['notebooks'] + 'config_sistema.json'

# Tenta carregar configuração existente primeiro
loaded_config_data = load_config(config_file_path)

# Configurações de rate limiting para API (usa valores carregados ou define novos)
CONFIG = loaded_config_data.get('config', {}).get('rate_limiting', {
    'delay_between_requests': 2.5,  # segundos entre requisições
    'max_requests_per_minute': 25,  # limite seguro
    'max_requests_per_day': 1400,   # limite diário
    'timeout_request': 30,          # timeout por requisição
    'max_retries': 3,              # tentativas em caso de erro
    'backoff_factor': 2,           # fator de backoff exponencial
})

# Configurações de checkpoint (usa valores carregados ou define novos)
CHECKPOINT_CONFIG = loaded_config_data.get('config', {}).get('checkpoint', {
    'interval': 25,                # salvar checkpoint a cada N artigos
    'directory': 'checkpoints/',   # diretório para checkpoints
    'prefix': 'checkpoint_',       # prefixo dos arquivos
    'max_files': 10,              # máximo de arquivos de checkpoint
})

# Configurações de caminhos (usa valores carregados ou define novos, mas estes são mais fixos)
PATHS = loaded_config_data.get('paths', {
    'data_raw': '/home/delon/Modelos/modeloCenanoInk/data/raw/',
    'excel_files': '/home/delon/Modelos/modeloCenanoInk/data/raw/arquivos_excel_artigo_cienciometrico/',
    'processed': '/home/delon/Modelos/modeloCenanoInk/data/processed/',
    'results': '/home/delon/Modelos/modeloCenanoInk/results/',
    'notebooks': '/home/delon/Modelos/modeloCenanoInk/notebooks/'
})

# Criar diretórios se não existirem (importante garantir que PATHS esteja definido)
for path_name, path_value in PATHS.items():
    Path(path_value).mkdir(parents=True, exist_ok=True)

# Configurações de análise (usa valores carregados ou define novos)
ANALYSIS_CONFIG = loaded_config_data.get('config', {}).get('analysis', {
    'min_abstract_length': 50,     # tamanho mínimo do abstract
    'score_threshold_high': 5,     # threshold para relevância alta
    'score_threshold_medium': 2,   # threshold para relevância média
    'use_title_boost': True,       # dar peso extra para palavras no título
    'title_boost_factor': 2,       # fator de multiplicação para título
})

print("⚙️ Configurações do sistema processadas (carregadas ou definidas):")
print(f"  🔄 Rate limiting: {CONFIG.get('delay_between_requests', 'N/A')}s entre requisições")
print(f"  💾 Checkpoints: a cada {CHECKPOINT_CONFIG.get('interval', 'N/A')} artigos")
print(f"  📁 Diretórios: {len(PATHS)} caminhos configurados")
print(f"  🎯 Análise: thresholds {ANALYSIS_CONFIG.get('score_threshold_medium', 'N/A')}/{ANALYSIS_CONFIG.get('score_threshold_high', 'N/A')}")

# 🧪 Validação do Ambiente

def validar_ambiente() -> Dict[str, bool]:
    """Valida se o ambiente está pronto para execução"""
    
    print("🧪 Validando ambiente do sistema...")
    
    validacao = {
        'api_configurada': False,
        'diretorios_existem': False,
        'excel_files_encontrados': False,
        'dependencias_instaladas': False,
        'espaco_disco_suficiente': False
    }
    
    # 1. Verificar API
    validacao['api_configurada'] = API_CONFIGURADA
    status_api = "✅" if API_CONFIGURADA else "❌"
    print(f"  {status_api} API Gemini: {'Configurada' if API_CONFIGURADA else 'Não configurada'}")
    
    # 2. Verificar diretórios
    dirs_ok = all(os.path.exists(path) for path in PATHS.values())
    validacao['diretorios_existem'] = dirs_ok
    status_dirs = "✅" if dirs_ok else "❌"
    print(f"  {status_dirs} Diretórios: {'Todos existem' if dirs_ok else 'Alguns faltando'}")
    
    # 3. Verificar arquivos Excel
    try:
        excel_files = list(Path(PATHS['excel_files']).glob('*.xls'))
        validacao['excel_files_encontrados'] = len(excel_files) > 0
        status_excel = "✅" if len(excel_files) > 0 else "❌"
        print(f"  {status_excel} Arquivos Excel: {len(excel_files)} arquivos encontrados")
    except Exception as e:
        print(f"  ❌ Arquivos Excel: Erro ao verificar - {e}")
    
    # 4. Verificar dependências
    try:
        import pandas, google.generativeai, matplotlib, seaborn
        validacao['dependencias_instaladas'] = True
        print(f"  ✅ Dependências: Todas instaladas")
    except ImportError as e:
        print(f"  ❌ Dependências: Faltando - {e}")
    
    # 5. Verificar espaço em disco (simplificado)
    try:
        import shutil
        total, used, free = shutil.disk_usage(PATHS['data_raw'])
        free_gb = free // (1024**3)
        validacao['espaco_disco_suficiente'] = free_gb > 1  # pelo menos 1GB
        status_disk = "✅" if free_gb > 1 else "❌"
        print(f"  {status_disk} Espaço em disco: {free_gb:.1f}GB disponível")
    except Exception as e:
        print(f"  ⚠️ Espaço em disco: Não foi possível verificar")
        validacao['espaco_disco_suficiente'] = True  # assumir ok
    
    # Resumo
    total_checks = len(validacao)
    passed_checks = sum(validacao.values())
    
    print(f"\n📊 Resultado da validação: {passed_checks}/{total_checks} verificações aprovadas")
    
    if passed_checks == total_checks:
        print("🎉 Ambiente completamente pronto!")
    elif passed_checks >= total_checks - 1:
        print("⚠️ Ambiente quase pronto - algumas configurações opcionais faltando")
    else:
        print("❌ Ambiente com problemas - configurações necessárias")
    
    return validacao

# Executar validação
validacao_resultado = validar_ambiente()

✅ Configuração carregada de: /home/delon/Modelos/modeloCenanoInk/notebooks/config_sistema.json
⚙️ Configurações do sistema processadas (carregadas ou definidas):
  🔄 Rate limiting: 2.5s entre requisições
  💾 Checkpoints: a cada 25 artigos
  📁 Diretórios: 5 caminhos configurados
  🎯 Análise: thresholds 2/5
🧪 Validando ambiente do sistema...
  ✅ API Gemini: Configurada
  ✅ Diretórios: Todos existem
  ✅ Arquivos Excel: 39 arquivos encontrados


  ✅ Dependências: Todas instaladas
  ✅ Espaço em disco: 3.0GB disponível

📊 Resultado da validação: 5/5 verificações aprovadas
🎉 Ambiente completamente pronto!


In [7]:
# 🏁 Finalização da Configuração

# Marcar sistema como configurado
SISTEMA_CONFIGURADO = True

# Preparar dados para salvar
config_data_to_save = {
    'sistema_configurado': SISTEMA_CONFIGURADO,
    'api_configurada': API_CONFIGURADA,
    'timestamp_configuracao': datetime.now().isoformat(),
    'validacao': validacao_resultado,
    'config': { # Agrupando as sub-configurações
        'rate_limiting': CONFIG,
        'checkpoint': CHECKPOINT_CONFIG,
        'analysis': ANALYSIS_CONFIG
    },
    'paths': PATHS,
    'nanomateriais': PATTERNS_NANOMATERIAIS, 
    'palavras_chave': PATTERNS_PALAVRAS_CHAVE 
}

# Salvar configuração usando a nova função helper
config_file_final_path = PATHS['notebooks'] + 'config_sistema.json'
save_config(config_data_to_save, config_file_final_path)

print("🏁 Configuração finalizada!")
# A mensagem de sucesso do save_config já informa o caminho
# print(f"💾 Configuração salva em: {config_file_final_path}") 

print("\n📋 Variáveis globais disponíveis:")
print("  - SISTEMA_CONFIGURADO:", SISTEMA_CONFIGURADO)
print("  - API_CONFIGURADA:", API_CONFIGURADA)
print("  - PATTERNS_NANOMATERIAIS: Base de nanomateriais (incluída no JSON)")
print("  - PATTERNS_PALAVRAS_CHAVE: Palavras-chave funcionais (incluídas no JSON)")
print("  - CONFIG: Configurações de rate limiting")
print("  - CHECKPOINT_CONFIG: Configurações de checkpoint")
print("  - ANALYSIS_CONFIG: Configurações de análise")
print("  - PATHS: Caminhos do projeto")

print("\n🚀 Próximo passo: Execute o notebook '02_carregamento_dados.ipynb'")

# Criar arquivo .gitignore se não existir
gitignore_content = '''
# API Keys e arquivos sensíveis
api_key
api_key.txt
*.key
.env

# Arquivos de checkpoint e resultados temporários
checkpoints/
*checkpoint*.csv
temp_*.csv

# Configurações locais
# config_sistema.json # Comentado para permitir versionamento se desejado, mas geralmente é melhor ignorar

# Jupyter
.ipynb_checkpoints/
'''

gitignore_path = Path(PATHS.get('notebooks', '.')).parent / '.gitignore' # Usar .get com fallback
if not gitignore_path.exists() and PATHS.get('notebooks'): # Adicionar verificação se PATHS['notebooks'] existe
    try:
        with open(gitignore_path, 'w', encoding='utf-8') as f:
            f.write(gitignore_content)
        print("🔒 Arquivo .gitignore criado/atualizado para proteger dados sensíveis")
    except Exception as e:
        print(f"⚠️ Não foi possível criar/atualizar .gitignore: {e}")
else:
    if gitignore_path.exists():
        print("ℹ️ Arquivo .gitignore já existe.")
    else:
        print("⚠️ Não foi possível determinar o local para .gitignore pois PATHS['notebooks'] não está definido.")

💾 Configuração salva com sucesso em: /home/delon/Modelos/modeloCenanoInk/notebooks/config_sistema.json
🏁 Configuração finalizada!

📋 Variáveis globais disponíveis:
  - SISTEMA_CONFIGURADO: True
  - API_CONFIGURADA: True
  - PATTERNS_NANOMATERIAIS: Base de nanomateriais (incluída no JSON)
  - PATTERNS_PALAVRAS_CHAVE: Palavras-chave funcionais (incluídas no JSON)
  - CONFIG: Configurações de rate limiting
  - CHECKPOINT_CONFIG: Configurações de checkpoint
  - ANALYSIS_CONFIG: Configurações de análise
  - PATHS: Caminhos do projeto

🚀 Próximo passo: Execute o notebook '02_carregamento_dados.ipynb'
ℹ️ Arquivo .gitignore já existe.
